Import Libraries

In [116]:
import math
import pandas as pd
import pickle
import os
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords, wordnet as wn
from nltk.stem import WordNetLemmatizer
from itertools import chain


In [117]:
custom_path = os.path.join(os.getcwd(), 'nltk_data')  #  in project dir
if not os.path.exists(custom_path):
    os.makedirs(custom_path)
nltk.data.path.append(custom_path)

nltk.download('punkt', download_dir=custom_path)
nltk.download('wordnet', download_dir=custom_path)
nltk.download('stopwords', download_dir=custom_path)

[nltk_data] Downloading package punkt to
[nltk_data]     c:\Users\NINJA\Documents\ITU\Semester7\Information
[nltk_data]     Retrieval\Homework-Assignment-3\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     c:\Users\NINJA\Documents\ITU\Semester7\Information
[nltk_data]     Retrieval\Homework-Assignment-3\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     c:\Users\NINJA\Documents\ITU\Semester7\Information
[nltk_data]     Retrieval\Homework-Assignment-3\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

Read the file in pandas dataframe

In [118]:
df = pd.read_csv('Articles.csv', encoding='ISO-8859-1') 

In [119]:
df.columns

Index(['Article', 'Date', 'Heading', 'NewsType'], dtype='object')

In [120]:
df.head()

Article  \
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

Load Preprocessed Data

In [121]:
with open('preprocessed_data.pkl', 'rb') as f:
    df = pickle.load(f)

with open('inverted_index.pkl', 'rb') as f:
    inverted_index = pickle.load(f)

with open('dates_min_max.pkl', 'rb') as f:
    min_date, max_date = pickle.load(f)


Setup NLTK Resources

In [122]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()


Preprocess function

In [123]:
def preprocess(text):
    tokens = word_tokenize(text.lower())
    tokens = [lemmatizer.lemmatize(token) for token in tokens if token.isalnum() and token not in stop_words]
    return tokens


BM25 Parameters and Tokens Preparation

In [124]:
k1 = 1.5
b = 0.75
heading_boost = 2.0
first_sentence_boost = 1.5

df['tokens'] = df['processed_heading'] + df['processed_article']
tokens_list = df['tokens'].tolist()
doc_lengths = [len(doc) for doc in tokens_list]
avg_doc_length = sum(doc_lengths) / len(doc_lengths)


In [125]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
df[['Heading']].head(50)

,Heading
0,sindh govt decides to cut public transport fares by 7pc kti rej
1,asia stocks up in new year trad
2,hong kong stocks open 0.66 percent lower
3,asian stocks sink euro near nine year
4,us oil prices slip below 50 a barr
5,oil hits new 5.5 year lows as saudis defend
6,bullish kse jumps over 33000 psychological barrier
7,oil falls further in asian trad
8,sugar prices drop to rs 49.80 in sind
9,oil extends losses as world bank cuts growth for


BM25 Scoring Functions

In [126]:
def bm25_score(term, doc_id):
    if term not in inverted_index or doc_id not in inverted_index[term]:
        return 0

    tf = len(inverted_index[term][doc_id])
    df_term = len(inverted_index[term])
    N = len(df)

    idf = math.log((N - df_term + 0.5) / (df_term + 0.5) + 1)
    dl = doc_lengths[doc_id]

    numerator = tf * (k1 + 1)
    denominator = tf + k1 * (1 - b + b * (dl / avg_doc_length))
    score = idf * (numerator / denominator)

    if term in df.iloc[doc_id]['processed_heading']:
        score *= heading_boost
    if term in df.iloc[doc_id]['processed_article'][:50]:
        score *= first_sentence_boost

    return score

def bm25_total_score(query_terms, doc_id):
    return sum(bm25_score(t, doc_id) for t in query_terms)


Recency Scoring

In [127]:
def recency_score(date):
    if pd.isnull(date):
        return 0
    delta = (max_date - min_date).days
    if delta == 0:
        return 1
    return (date - min_date).days / delta


Query Expansion Functions

In [128]:
def get_wordnet_synonyms(term):
    syns = set()
    for syn in wn.synsets(term):
        for lemma in syn.lemmas():
            syns.add(lemma.name().lower().replace("_", " "))
    return syns

domain_synonyms = {
    "stock market": ["kse", "share market", "bourse", "equities"],
    "oil prices": ["crude", "petroleum", "oil market"],
    "team player": ["player", "athlete", "cricketer", "footballer"],
    "cricket match": ["odi", "t20", "test match", "cricket game"],
    "business economy": ["economic", "finance", "market", "trade"]
}

def expand_terms(query_terms):
    expanded = set(query_terms)
    for t in query_terms:
        expanded.update(get_wordnet_synonyms(t))
        if t in domain_synonyms:
            expanded.update(domain_synonyms[t])
    return list(expanded)


Weak Supervision Functions

In [129]:
def base_bm25(query_terms, top_n=200):
    scores = []
    for doc_id in df.index:
        s = sum(bm25_score(t, doc_id) for t in query_terms)
        scores.append((doc_id, s))
    scores = sorted(scores, key=lambda x: x[1], reverse=True)
    return [d for d, s in scores[:top_n]]

def build_relevant_docs(query, category):
    terms = preprocess(query)
    expanded = expand_terms(terms)

    bm25_candidates = set(base_bm25(terms, top_n=200))
    overlap_candidates = set()
    for doc_id in df.index[df['category'] == category]:
        doc_tokens = df.iloc[doc_id]['tokens']
        overlap = len(set(expanded).intersection(doc_tokens))
        if overlap >= 2:
            overlap_candidates.add(doc_id)

    rel = bm25_candidates.union(overlap_candidates)
    return rel

sample_queries = [
    {"query": "stock market", "category": "business"},
    {"query": "oil prices", "category": "business"},
    {"query": "kse index", "category": "business"},
    {"query": "team player", "category": "sports"},
    {"query": "cricket match", "category": "sports"},
    {"query": "business economy", "category": "business"}
]

relevant_docs = {}
for q in sample_queries:
    relevant_docs[q['query']] = build_relevant_docs(q['query'], q['category'])


Query Processing Function

In [130]:
def process_query(query, category_filter=None, k=10):
    query_terms = preprocess(query)
    expanded_terms = expand_terms(query_terms)

    if category_filter:
        candidate_docs = df.index[df['category'] == category_filter].tolist()
    else:
        candidate_docs = df.index.tolist()

    phrase_tokens = query.lower().split()
    phrase_bonus = {}
    for doc_id in candidate_docs:
        doc_tokens = tokens_list[doc_id]
        for i in range(len(doc_tokens) - len(phrase_tokens) + 1):
            match = True
            j_offset = 0
            for j, token in enumerate(phrase_tokens):
                if i + j + j_offset >= len(doc_tokens):
                    match = False
                    break
                if doc_tokens[i + j + j_offset] != token:
                    if i + j + j_offset + 1 < len(doc_tokens) and doc_tokens[i + j + j_offset + 1] == token:
                        j_offset += 1
                    else:
                        match = False
                        break
            if match:
                phrase_bonus[doc_id] = 2.0
                break

    scored_docs = []
    for d in candidate_docs:
        score_bm25 = bm25_total_score(expanded_terms, d)
        score_recency = recency_score(df.iloc[d]['Date']) * 0.5
        score_phrase = phrase_bonus.get(d, 0)
        overlap_bonus = len(set(expanded_terms).intersection(df.iloc[d]['tokens'])) * 0.2
        final_score = score_bm25 + score_recency + score_phrase + overlap_bonus
        scored_docs.append((d, final_score))

    ranked = sorted(scored_docs, key=lambda x: x[1], reverse=True)
    return [(df.iloc[d]['Heading'], df.iloc[d]['Date']) for d, _ in ranked[:k]]


Recall Calculation

In [131]:
def recall_at_10(query, category):
    top_docs = process_query(query, category, k=10)
    rel_docs = relevant_docs.get(query, [])

    if len(rel_docs) == 0:
        return None
    top_ids = [d for d, _ in enumerate(df['Heading']) if df.iloc[d]['Heading'] in [h for h, _ in top_docs]]
    retrieved_relevant = len(set(top_ids) & set(rel_docs))
    recall = retrieved_relevant / len(rel_docs)
    return recall


Test

In [132]:
for q in sample_queries:
    r = recall_at_10(q["query"], q["category"])
    print(f"Query: {q['query']}, Recall@10: {r}")

print("Top results for 'stock market':", process_query("stock market", "business"))
print("Top results for 'cricket match':", process_query("cricket match", "sports"))


Query: stock market, Recall@10: 0.020833333333333332
Query: oil prices, Recall@10: 0.023529411764705882
Query: kse index, Recall@10: 0.03333333333333333
Query: team player, Recall@10: 0.02967359050445104
Query: cricket match, Recall@10: 0.017699115044247787
Query: business economy, Recall@10: 0.030959752321981424
Top results for 'stock market': [('Pakistans stock market dominates Asia Bloomberg', Timestamp('2016-06-15 00:00:00')), ('oil prices fall before us inventory d', Timestamp('2015-10-15 00:00:00')), ('china stock market freezing up as sell off gathers ', Timestamp('2015-07-08 00:00:00')), ('Oil dips on rising crude inventories after Trumps shock victory', Timestamp('2016-11-10 00:00:00')), ('oil prices fall on eve of us inventory d', Timestamp('2015-02-19 00:00:00')), ('Oil prices edge up despite unexpected US crude inventory build', Timestamp('2016-12-30 00:00:00')), ('Pakistani stocks end higher ahead of MSCI annou', Timestamp('2016-06-13 00:00:00')), ('Gold climbs as stock ra